In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import pycountry

#### Import ```attacks.csv```

In [43]:
pd.set_option ("display.max_columns", None)
df = pd.read_csv("data/attacks.csv", encoding="latin1")
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

# CLEANING 


#### Remove the ```columns``` i don't need

In [45]:
df.drop(columns=["Investigator or Source", 
                 "pdf", 
                 "href formula", 
                 "href", 
                 "Case Number.1", 
                 "Case Number.2", 
                 "original order", 
                 "Unnamed: 22",
                 "Unnamed: 23",
                 "Name",
                 "Location",
                 "Case Number",
                 "Area",
                 "Age",
                 "Sex ",
                 "Time",
                 "Fatal (Y/N)",
                 "Type",
                 "Activity",
                 "Injury"], 
        axis=1, inplace=True)
df

,Date,Year,Country,Species
0,25-Jun-2018,2018.0,USA,White shark
1,18-Jun-2018,2018.0,USA,NaN
2,09-Jun-2018,2018.0,USA,NaN
3,08-Jun-2018,2018.0,AUSTRALIA,2 m shark
4,04-Jun-2018,2018.0,MEXICO,"Tiger shark, 3m"
...,...,...,...,...
25718,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN


#### Remove the ```rows``` i don't need

In [46]:
df.dropna(axis = 0, how = 'all', inplace = True)
df

,Date,Year,Country,Species
0,25-Jun-2018,2018.0,USA,White shark
1,18-Jun-2018,2018.0,USA,NaN
2,09-Jun-2018,2018.0,USA,NaN
3,08-Jun-2018,2018.0,AUSTRALIA,2 m shark
4,04-Jun-2018,2018.0,MEXICO,"Tiger shark, 3m"
...,...,...,...,...
6297,Before 1903,0.0,AUSTRALIA,NaN
6298,Before 1903,0.0,AUSTRALIA,NaN
6299,1900-1905,0.0,USA,NaN
6300,1883-1889,0.0,PANAMA,NaN


# TRANSFORMING


## Hipothesis 1: Are sharks more aggressive according to the season of the year?
    Seasons: 
        To get the seasons i need first to sort the countries in the "Country" column by hemisphere.
        - Winter: Dec(12), Jan(1), Feb(2)
        - Spring: Mar(3), Apr(4), May(5)
        - Summer: Jun(6), Jul(7), Aug(8)
        - Autumn: Sep(9), Oct(10), Nov(11)
    Type Sharks:
        - White Shark: WHITE
        - Bull Shark: BULL
        - Mako Shark: MAKO
        - Tiger Shark: TIGER
        - Blacktip Shark: BLACKTIP

### SEASONS

#### 01. Change the name of the countries in the column "Country" to capitalize the first word of each word.
        In this way, pycountry will detect the countries properly.

In [47]:
# First we remove the NaN values from the Country column, otherwise will get an error trying to capitalize floats.
df.dropna(subset=['Country'], inplace=True) 

# Then we can use .apply method
df['Country'] = df['Country'].apply(lambda x: ' '.join([word.capitalize() for word in x.split()]))
df


,Date,Year,Country,Species
0,25-Jun-2018,2018.0,Usa,White shark
1,18-Jun-2018,2018.0,Usa,NaN
2,09-Jun-2018,2018.0,Usa,NaN
3,08-Jun-2018,2018.0,Australia,2 m shark
4,04-Jun-2018,2018.0,Mexico,"Tiger shark, 3m"
...,...,...,...,...
6297,Before 1903,0.0,Australia,NaN
6298,Before 1903,0.0,Australia,NaN
6299,1900-1905,0.0,Usa,NaN
6300,1883-1889,0.0,Panama,NaN


#### 02. Now we want to create a ```for loop``` to check if the elements of the ```column "Country"``` are detected by ```pycountry```.

In [48]:
valid_countries = []
for country in df['Country']:
    if pycountry.countries.get(name=country) is not None:
        valid_countries.append(country)

#### 03. Now we want the ```column "Country"``` to have only the values of the list ```valid_countries```.

In [49]:
df = df[df['Country'].isin(valid_countries)] #  using isin you can filter the rows according to whether the values belong to the valid_countries list or not.
df

,Date,Year,Country,Species
3,08-Jun-2018,2018.0,Australia,2 m shark
4,04-Jun-2018,2018.0,Mexico,"Tiger shark, 3m"
5,03-Jun-2018,2018.0,Australia,NaN
6,03-Jun-2018,2018.0,Brazil,Tiger shark
10,24-May-2018,2018.0,Australia,Grey reef shark
...,...,...,...,...
6295,Before 1906,0.0,Australia,Blue pointers
6296,Before 1906,0.0,Australia,Said to involve a grey nurse shark that leapt ...
6297,Before 1903,0.0,Australia,NaN
6298,Before 1903,0.0,Australia,NaN


In [50]:
df["Country"].unique()

array(['Australia', 'Mexico', 'Brazil', 'South Africa', 'Thailand',
       'Costa Rica', 'Maldives', 'Bahamas', 'New Caledonia', 'Ecuador',
       'Malaysia', 'Libya', 'Cuba', 'Mauritius', 'New Zealand', 'Spain',
       'Samoa', 'Solomon Islands', 'Japan', 'Egypt', 'Comoros',
       'French Polynesia', 'United Kingdom', 'United Arab Emirates',
       'Philippines', 'Indonesia', 'China', 'Fiji', 'Dominican Republic',
       'Cayman Islands', 'Aruba', 'Mozambique', 'Puerto Rico', 'Italy',
       'Greece', 'France', 'Papua New Guinea', 'Kiribati', 'Israel',
       'Jamaica', 'Guam', 'Seychelles', 'Belize', 'Nigeria', 'Tonga',
       'Canada', 'Croatia', 'Saudi Arabia', 'Chile', 'Kenya', 'Malta',
       'Madagascar', 'Panama', 'Somalia', 'Norway', 'Senegal', 'Yemen',
       'Sierra Leone', 'Liberia', 'Vanuatu', 'Honduras', 'Sri Lanka',
       'Uruguay', 'India', 'Marshall Islands', 'Hong Kong', 'El Salvador',
       'Angola', 'Bermuda', 'Montenegro', 'Tunisia', 'Namibia',
       'Portugal'

#### 04. Now we want to define a ```function``` to check if the countries are from ```North``` or ```South``` hemisphere with ```pycountry```

def get_hemisphere(country):
    try:
        info = pycountry.countries.get(name=country)
        lat = info.latitude

        if lat >= 0:
            return "North"
        else:
            return "South"
    except:
        return "Error"

### MONTHS / SEASONS

#### 01. I need to create a ```column``` with the ```months``` of the year based on the column "Date"

In [54]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce') # Convert the 'Date' column to datetime format.
df['Month'] = df['Date'].dt.month # Create a column 'Month' with the values of the months
#df['Day'] = df['Date'].dt.day # Create a column 'Day' with the values of the days
df

,Date,Year,Country,Species,Month,Seasons
3,2018-06-08,2018.0,Australia,2 m shark,6,Summer
4,2018-06-04,2018.0,Mexico,"Tiger shark, 3m",6,Summer
5,2018-06-03,2018.0,Australia,NaN,6,Summer
6,2018-06-03,2018.0,Brazil,Tiger shark,6,Summer
10,2018-05-24,2018.0,Australia,Grey reef shark,5,Spring
...,...,...,...,...,...,...
6124,1807-01-12,1807.0,Australia,NaN,1,Winter
6136,1788-05-10,1788.0,Australia,NaN,5,Spring
6140,1784-12-12,1784.0,Jamaica,NaN,12,Winter
6141,1783-03-02,1783.0,Martinique,NaN,3,Spring


#### 03. I need to create a ```column``` with the ```seasons``` of the year grouping the months in the column "Month"

In [52]:
def season_function(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'No Season'

df['Seasons'] = df['Month'].apply(season_function)
df = df.drop(df[df['Seasons'] == 'No Season'].index)
df

/var/folders/d2/c8nbsrk920lgrdq2nn0mv1t00000gn/T/ipykernel_11466/2948494714.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Seasons'] = df['Month'].apply(season_function)


,Date,Year,Country,Species,Month,Seasons
3,2018-06-08,2018.0,Australia,2 m shark,6.0,Summer
4,2018-06-04,2018.0,Mexico,"Tiger shark, 3m",6.0,Summer
5,2018-06-03,2018.0,Australia,NaN,6.0,Summer
6,2018-06-03,2018.0,Brazil,Tiger shark,6.0,Summer
10,2018-05-24,2018.0,Australia,Grey reef shark,5.0,Spring
...,...,...,...,...,...,...
6124,1807-01-12,1807.0,Australia,NaN,1.0,Winter
6136,1788-05-10,1788.0,Australia,NaN,5.0,Spring
6140,1784-12-12,1784.0,Jamaica,NaN,12.0,Winter
6141,1783-03-02,1783.0,Martinique,NaN,3.0,Spring


### SPECIES

#### 01. I need to ```remove null values``` from "Month"column

In [56]:
list(df['Species '].unique())

['2 m shark',
 'Tiger shark, 3m',
 nan,
 'Tiger shark',
 'Grey reef shark',
 'Shark involvement not confirmed',
 'Questionable',
 '3 m shark',
 'White shark, 3.5 m',
 'White shark, 2.5 m',
 'Juvenile bull shark',
 'Bull shark',
 'White shark',
 'Wobbegong shark',
 '3.5 m shark',
 '1.8 m shark',
 'Blacktip shark',
 'Juvenile white shark,  2.7 to 3.2 m',
 'Bull shark, 2 m',
 'Possibly a wobbegong',
 'Injury believed caused by an eel, not a shark',
 'Galapagos shark?',
 '2m shark',
 'Bull shark, 3 m ',
 'Grey reef shark. 2 m',
 'small shark',
 'Wobbegong shark?',
 'Juvenile nurse shark',
 "Nurse shark. 5'",
 'White shark, 3 m',
 'Shark involvement questionable',
 "5' shark",
 'Oceanic whitetip shark, 1.8 to 2 m',
 "Blue shark 6'",
 'Shark involvement prior to death not confirmed',
 'White shark, 4 m',
 'Seven-gill shark',
 "10' shark",
 'Bull shark, 1.5',
 'Lemon shark pup',
 'Caribbean reef shark',
 'Bull shark, 4 m',
 'Bull shark, 1 m',
 'a small shark',
 'Wobbegong shark, 2m',
 'Blue s

In [57]:
#sns.countplot(x=df_months["Month"], palette="magma");

01. I need to create a column with the months of the year based on the column "Date"

In [58]:
df['Species'] = df['Species '].str.replace(r'.*bull.*', 'BULL', case=False, regex=True) 
df['Species'] = df['Species '].str.replace(r'.*mako.*', 'MAKO', case=False, regex=True)
df['Species'] = df['Species '].str.replace(r'.*white shark.*', 'WHITE SHARK', case=False, regex=True)
df['Species'] = df['Species '].str.replace(r'.*tiger.*', 'TIGER', case=False, regex=True)
df['Species'] = df['Species '].str.replace(r'.*blacktip.*', 'BLACKTIP', case=False, regex=True)
df

,Date,Year,Country,Species,Month,Seasons,Species
3,2018-06-08,2018.0,Australia,2 m shark,6,Summer,2 m shark
4,2018-06-04,2018.0,Mexico,"Tiger shark, 3m",6,Summer,"Tiger shark, 3m"
5,2018-06-03,2018.0,Australia,NaN,6,Summer,NaN
6,2018-06-03,2018.0,Brazil,Tiger shark,6,Summer,Tiger shark
10,2018-05-24,2018.0,Australia,Grey reef shark,5,Spring,Grey reef shark
...,...,...,...,...,...,...,...
6124,1807-01-12,1807.0,Australia,NaN,1,Winter,NaN
6136,1788-05-10,1788.0,Australia,NaN,5,Spring,NaN
6140,1784-12-12,1784.0,Jamaica,NaN,12,Winter,NaN
6141,1783-03-02,1783.0,Martinique,NaN,3,Spring,NaN


In [59]:
shark_species = ['MAKO', 'TIGER', 'WHITE SHARK', 'BULL', 'BLACKTIP']

df[df['Species '].isin(shark_species)]

,Date,Year,Country,Species,Month,Seasons,Species


In [60]:
df.dropna(subset=['Month'], inplace=True)
df

,Date,Year,Country,Species,Month,Seasons,Species
3,2018-06-08,2018.0,Australia,2 m shark,6,Summer,2 m shark
4,2018-06-04,2018.0,Mexico,"Tiger shark, 3m",6,Summer,"Tiger shark, 3m"
5,2018-06-03,2018.0,Australia,NaN,6,Summer,NaN
6,2018-06-03,2018.0,Brazil,Tiger shark,6,Summer,Tiger shark
10,2018-05-24,2018.0,Australia,Grey reef shark,5,Spring,Grey reef shark
...,...,...,...,...,...,...,...
6124,1807-01-12,1807.0,Australia,NaN,1,Winter,NaN
6136,1788-05-10,1788.0,Australia,NaN,5,Spring,NaN
6140,1784-12-12,1784.0,Jamaica,NaN,12,Winter,NaN
6141,1783-03-02,1783.0,Martinique,NaN,3,Spring,NaN


In [61]:
df_1 = df[["Month", "Species "]].dropna()
df_1

,Month,Species
3,6,2 m shark
4,6,"Tiger shark, 3m"
6,6,Tiger shark
10,5,Grey reef shark
16,5,Shark involvement not confirmed
...,...,...
6009,4,234-lb shark
6014,12,Shark involvement prior to death unconfirmed
6021,2,White sharks
6035,1,Shark involvement prior to death unconfirmed


In [ ]:
sns.countplot(x=df_1["Month"], hue=df_1["Species "], palette="magma");

/Users/begui/miniconda3/envs/ironhack/lib/python3.9/site-packages/IPython/core/events.py:89: UserWarning: Glyph 147 (\x93) missing from current font.
  func(*args, **kwargs)
/Users/begui/miniconda3/envs/ironhack/lib/python3.9/site-packages/IPython/core/events.py:89: UserWarning: Glyph 148 (\x94) missing from current font.
  func(*args, **kwargs)
/Users/begui/miniconda3/envs/ironhack/lib/python3.9/site-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 147 (\x93) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/Users/begui/miniconda3/envs/ironhack/lib/python3.9/site-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 148 (\x94) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)


## TIME


In [ ]:
 TIME


# DRAFT 


In [ ]:
list(df.columns)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.Country.unique()

In [ ]:
list(df.Time.unique()) # get the unique values

In [ ]:
list1= list(df.Country.unique())
list1

In [33]:
country_name = 'MID ATLANTIC OCEAN'
country_obj = pycountry.countries.get(name=country_name)
if country_obj is not None:
    print(f"{country_name} EXISTS in pycountry")
else:
    print(f"{country_name} does not exist in pycountry")

MID ATLANTIC OCEAN does not exist in pycountry


In [ ]:
df['Country'] = df['Country'].apply(lambda x: x.title())

# Filtering

In [ ]:
df.Year == 